## Imports

In [2]:
from dotenv import load_dotenv
load_dotenv()
from tqdm import tqdm
from langchain_community.document_loaders import DirectoryLoader, UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# nltk import
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/abhigyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Path configurations

In [5]:
doc_dir = '/home/abhigyan/Desktop/projects/rag_langchain_2026api/docs_dir'
vector_db_dir = '/home/abhigyan/Desktop/projects/rag_langchain_2026api/vector_db'
collection_name = 'textbook_collections'

## Embeddings model

In [3]:
embed_model = HuggingFaceEmbeddings()

Loading weights: 100%|██████████| 199/199 [00:01<00:00, 145.31it/s, Materializing param=pooler.dense.weight]                        
MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


## Loading the documents

In [6]:
# directory loader
loader = DirectoryLoader(doc_dir, glob='./*.pdf*', loader_cls=UnstructuredFileLoader)
documents = loader.load()

Cannot set non-stroke color because expected 4 components but got [0]
Cannot set non-stroke color because expected 4 components but got [0]


Cannot set non-stroke color because expected 4 components but got [0]


In [7]:
print(type(documents))
print(len(documents))


<class 'list'>
2


In [8]:
print(documents[0].page_content)

CHAPTER 12

ECOSYSTEM

12.1 Ecosystem–Structure and Function

12.2. Productivity

12.3 Decomposition

12.4 Energy Flow

12.5 Ecological Pyramids

An ecosystem can be visualised as a functional unit of nature, where living organisms interact among themselves and also with the surrounding physical environment. Ecosystem varies greatly in size from a small pond to a large forest or a sea. Many ecologists regard the entire biosphere as a global ecosystem, as a composite of all local ecosystems on Earth. Since this system is too much big and complex to be studied at one time, it is convenient to divide it into two basic categories, namely the terrestrial and the aquatic. Forest, grassland and desert are some examples of terrestrial ecosystems; pond, lake, wetland, river and estuary are some examples of aquatic ecosystems. Crop fields and an aquarium may also be considered as man-made ecosystems.

We will first look at the structure of the ecosystem, in order to appreciate the input (product

## Splitting and Chunking

In [9]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [10]:
text_chunks = text_splitter.split_documents(documents)

Created a chunk of size 1582, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1959, which is longer than the specified 1000
Created a chunk of size 1223, which is longer than the specified 1000
Created a chunk of size 1251, which is longer than the specified 1000
Created a chunk of size 1069, which is longer than the specified 1000
Created a chunk of size 1083, which is longer than the specified 1000
Created a chunk of size 1298, which is longer than the specified 1000
Created a chunk of size 1195, which is longer than the specified 1000
Created a chunk of size 1054, which is longer than the specified 1000


In [12]:
len(text_chunks)

63

In [15]:
text_chunks[0]

Document(metadata={'source': '/home/abhigyan/Desktop/projects/rag_langchain_2026api/docs_dir/12. Ecosystem.pdf'}, page_content='CHAPTER 12\n\nECOSYSTEM\n\n12.1 Ecosystem–Structure and Function\n\n12.2. Productivity\n\n12.3 Decomposition\n\n12.4 Energy Flow\n\n12.5 Ecological Pyramids\n\nAn ecosystem can be visualised as a functional unit of nature, where living organisms interact among themselves and also with the surrounding physical environment. Ecosystem varies greatly in size from a small pond to a large forest or a sea. Many ecologists regard the entire biosphere as a global ecosystem, as a composite of all local ecosystems on Earth. Since this system is too much big and complex to be studied at one time, it is convenient to divide it into two basic categories, namely the terrestrial and the aquatic. Forest, grassland and desert are some examples of terrestrial ecosystems; pond, lake, wetland, river and estuary are some examples of aquatic ecosystems. Crop fields and an aquarium m

## Storing chunks in VectorDB

In [16]:
vector_store = Chroma.from_documents(text_chunks, 
                                     embed_model, 
                                     collection_name=collection_name, 
                                     persist_directory=vector_db_dir)